# advent of code 2024 - [day 12](https://adventofcode.com/2024/day/12)

In [30]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']


In [31]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

import pandas as pd

2.13.0


In [32]:
queries = [
"CALL apoc.schema.assert({},{});",
"""MATCH (n)
CALL {WITH n DETACH DELETE n}
IN TRANSACTIONS OF 1000 ROWS;""",
"""CALL gds.graph.list()
YIELD graphName
WITH graphName AS g
CALL (g) {CALL gds.graph.drop(g) YIELD graphName RETURN graphName}
WITH graphName RETURN graphName;"""]

for q in queries:
    gds.run_cypher(q, {})


In [33]:
def gen_lists(file='input.txt'):
    """Generates tuples of integers"""
    file = open(file, 'r')
    for ix, line in enumerate(file):
        for jx, c in enumerate(line.strip()):
            yield ix, jx, c

In [34]:
filename = "input.txt"
#filename = "test.txt"
#filename = "test2.txt"

## Neo4j-based solution

### Parsing

In [35]:
tiles = [{'row':ix, 'col':jx, 'val':c} for ix, jx, c in list(gen_lists(filename))]

In [36]:
tiles

[{'row': 0, 'col': 0, 'val': 'G'},
 {'row': 0, 'col': 1, 'val': 'G'},
 {'row': 0, 'col': 2, 'val': 'G'},
 {'row': 0, 'col': 3, 'val': 'G'},
 {'row': 0, 'col': 4, 'val': 'G'},
 {'row': 0, 'col': 5, 'val': 'D'},
 {'row': 0, 'col': 6, 'val': 'D'},
 {'row': 0, 'col': 7, 'val': 'D'},
 {'row': 0, 'col': 8, 'val': 'D'},
 {'row': 0, 'col': 9, 'val': 'S'},
 {'row': 0, 'col': 10, 'val': 'S'},
 {'row': 0, 'col': 11, 'val': 'S'},
 {'row': 0, 'col': 12, 'val': 'E'},
 {'row': 0, 'col': 13, 'val': 'E'},
 {'row': 0, 'col': 14, 'val': 'E'},
 {'row': 0, 'col': 15, 'val': 'E'},
 {'row': 0, 'col': 16, 'val': 'E'},
 {'row': 0, 'col': 17, 'val': 'E'},
 {'row': 0, 'col': 18, 'val': 'E'},
 {'row': 0, 'col': 19, 'val': 'E'},
 {'row': 0, 'col': 20, 'val': 'E'},
 {'row': 0, 'col': 21, 'val': 'E'},
 {'row': 0, 'col': 22, 'val': 'E'},
 {'row': 0, 'col': 23, 'val': 'E'},
 {'row': 0, 'col': 24, 'val': 'E'},
 {'row': 0, 'col': 25, 'val': 'E'},
 {'row': 0, 'col': 26, 'val': 'E'},
 {'row': 0, 'col': 27, 'val': 'U'},
 {

### Building Lattice

In [37]:
gds.run_cypher('CREATE INDEX tile_row IF NOT EXISTS FOR (r:Tile) ON (r.row)')
gds.run_cypher('CREATE INDEX tile_col IF NOT EXISTS FOR (r:Tile) ON (r.col)')
gds.run_cypher('CREATE INDEX tile_val IF NOT EXISTS FOR (r:Tile) ON (r.val)')
gds.run_cypher('CREATE INDEX tile_comp IF NOT EXISTS FOR (r:Tile) ON (r.componentId)')
gds.run_cypher('CREATE INDEX tile_col IF NOT EXISTS FOR (r:Tile) ON (r.col, r.row, r.val)')
gds.run_cypher('CREATE INDEX port_comp IF NOT EXISTS FOR (p:Port) ON (p.row, p.col, p.dir)')
gds.run_cypher('CREATE INDEX has_port_dir FOR ()-[r:HAS_PORT]-() ON (r.dir)')


""


In [38]:
query_ingest = """
UNWIND $tiles AS tile
CREATE (:Tile {row:tile.row, col:tile.col, val:tile.val} )
"""

gds.run_cypher(query_ingest, {"tiles":tiles})

""


In [39]:
gds.run_cypher("""
MATCH (t:Tile)
WITH t.row AS row_num, t ORDER BY t.col
WITH row_num, collect(t) AS row
CALL apoc.nodes.link(row, 'EAST')
""")

""


In [40]:
gds.run_cypher("""
MATCH (t:Tile)
WITH t.col AS col_num, t ORDER BY t.row
WITH col_num, collect(t) AS col
CALL apoc.nodes.link(col, 'SOUTH')
""")

""


In [41]:
gds.run_cypher("""
MATCH (x)-[r:EAST|SOUTH]->(y)
WHERE x.val <> y.val
CALL (r) {
    DELETE r
} IN TRANSACTIONS OF 1000 ROWS
""")

""


In [42]:
gds.run_cypher("""
MATCH (source:Tile)
OPTIONAL MATCH path=(source)-[r:EAST|SOUTH]->(target)
WITH gds.graph.project(
  'myGraph',
  source,
  target,
  {}
) AS g
CALL gds.wcc.write('myGraph', { writeProperty: 'componentId' })
YIELD nodePropertiesWritten, componentCount
RETURN nodePropertiesWritten, componentCount
""")

,nodePropertiesWritten,componentCount
0,19600,620


In [43]:
gds.run_cypher("""
MATCH (n:Tile)
WITH n, count{(n)-[:SOUTH|EAST]-()} AS degree
WITH n.componentId AS comp, collect(degree) AS degrees
WITH comp, size(degrees) AS area, reduce(acc=0, d in degrees| acc+(4-d)) AS perimeter
RETURN sum(area * perimeter) AS part1
""")

,part1
0,1361494


In [44]:
gds.run_cypher("""
CYPHER runtime=parallel
CALL () {
MATCH path = (n:Tile)(
(i)-[:SOUTH]->(j)
WHERE NOT EXISTS {(j)-[:EAST]->()}
)*
(last)
WHERE NOT EXISTS {(n)-[:EAST]->()}
AND NOT EXISTS {(z WHERE NOT EXISTS {(z)-[:EAST]->()})-[:SOUTH]->(n)}
AND NOT EXISTS {(last)-[:SOUTH]->(z WHERE NOT EXISTS {(z)-[:EAST]->()})}
RETURN [n]+j AS nodes, "EAST" as fence_ori

UNION ALL

MATCH path = (n:Tile)(
(i)-[:SOUTH]->(j)
WHERE NOT EXISTS {(j)<-[:EAST]-()}
)*
(last)
WHERE NOT EXISTS {(n)<-[:EAST]-()}
AND NOT EXISTS {(z WHERE NOT EXISTS {(z)<-[:EAST]-()})-[:SOUTH]->(n)}
AND NOT EXISTS {(last)-[:SOUTH]->(z WHERE NOT EXISTS {(z)<-[:EAST]-()})}
RETURN [n]+j AS nodes, "WEST" as fence_ori

UNION ALL

MATCH path = (n:Tile)(
(i)-[:EAST]->(j)
WHERE NOT EXISTS {(j)-[:SOUTH]->()}
)*
(last)
WHERE NOT EXISTS {(n)-[:SOUTH]->()}
AND NOT EXISTS {(z WHERE NOT EXISTS {(z)-[:SOUTH]->()})-[:EAST]->(n)}
AND NOT EXISTS {(last)-[:EAST]->(z WHERE NOT EXISTS {(z)-[:SOUTH]->()})}
RETURN [n]+j AS nodes, "SOUTH" as fence_ori

UNION ALL

MATCH path = (n:Tile)(
(i)-[:EAST]->(j)
WHERE NOT EXISTS {(j)<-[:SOUTH]-()}
)*
(last)
WHERE NOT EXISTS {(n)<-[:SOUTH]-()}
AND NOT EXISTS {(z WHERE NOT EXISTS {(z)<-[:SOUTH]-()})-[:EAST]->(n)}
AND NOT EXISTS {(last)-[:EAST]->(z WHERE NOT EXISTS {(z)<-[:SOUTH]-()})}
RETURN [n]+j AS nodes, "NORTH" as fence_ori
}
WITH nodes, fence_ori
WITH nodes[0].componentId AS comp, count(fence_ori) AS fences
WITH comp, fences, count{(:Tile {componentId: comp})} AS area
RETURN sum (fences * area) AS part2
""")

,part2
0,830516
